In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import time
from selenium import webdriver
import csv
import requests
import secrets
from selenium.webdriver.common.keys import Keys


In [7]:
import os
import time
from selenium import webdriver
import json
from threading import Thread
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

visited_links = []
product_list = list()
field_names = ("current_category", "filters", "product_links")

links_filepath = "input.txt"
output_filepath = "output"

sub_categories_class_name = "_2SqgSY"
total_categories_class_name = "_2aDURW"
category_in_focus_class_name = "_1jJQdf"

def read_url():
    with open(links_filepath, "r", encoding="utf-8") as file:
        urls = file.read()
        urls = urls.split("\n")

    return urls

def check_exists_by_class_name(driver, class_name):
    try:
        driver.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

def re_scrape(driver, root):
    
    global tab_in_focus, product_list, visited_links
    
    if driver.current_url not in visited_links or root==True:
        visited_links.append(driver.current_url)
        try:
            if "more" in driver.find_elements_by_class_name(sub_categories_class_name)[-1].text:
                driver.find_elements_by_class_name(sub_categories_class_name)[-1].click()
        except:
            pass

        total_categories = []
        sub_categories = []
        sub_categories_links = []
        category_in_focus = ""

        try:
            total_categories = driver.find_element_by_class_name(total_categories_class_name).text.replace("CATEGORIES","").strip().split("\n")
            sub_categories = [ item.text for item in driver.find_elements_by_class_name(sub_categories_class_name) ]
            sub_categories_links = [ item.get_attribute('href') for item in driver.find_elements_by_class_name(sub_categories_class_name) ]
            category_in_focus = driver.find_element_by_class_name(category_in_focus_class_name).text
        except:
            pass
#         print(category_in_focus, driver.current_url)
        if len(sub_categories) > 0:
            for s_cat, link in zip(sub_categories, sub_categories_links):
                if s_cat != "Show less":
                    tab_in_focus+=1
                    # Open a new window
                    driver.execute_script("window.open('');")
                    # Switch to the new window
                    driver.switch_to.window(driver.window_handles[tab_in_focus])
    #                 print(tab_in_focus*" ", tab_in_focus, s_cat, "->", link)
                    driver.get(link)
                    re_scrape(driver, 0)

        current_category = ""
        filters = []
        product_links = []

        try:
            current_category = "->".join(total_categories[:total_categories.index(category_in_focus)+1])
        except:
            pass

        try:
            for item in driver.find_elements_by_class_name("ttx38n"):
                if item.get_attribute("class") == "ttx38n":
                    item.click()
        except:
            pass

        try:
            for item in driver.find_elements_by_class_name("QvtND5")[::-1]:
                if "MORE" in item.text:
                    driver.execute_script("arguments[0].click();", item)
        except:
            pass

        try:
            filters = [ filter_item.text.split("\n") for filter_item in driver.find_elements_by_class_name("_2hbLCH")[1:] ]
        except:
            pass

        try:
            product_class_name = "IRpwTa" if check_exists_by_class_name(driver, "IRpwTa") else "s1Q9rs"
            product_links = [ item.get_attribute('href') for item in driver.find_elements_by_class_name(product_class_name) ]
        except:
            pass

        dict_t =  {
            "current_category": current_category,
            "filters": filters,
            "product_links": product_links
        }

        product_list.append(dict_t.copy())
    
    if tab_in_focus != 0:
        driver.close()
        tab_in_focus-=1
        driver.switch_to.window(driver.window_handles[tab_in_focus])
    
    return True

if __name__ == "__main__":
    links_filepath = input("Enter links filepath : ")
    output_filepath = input("Enter output folder : ")
    
    if not os.path.exists(output_filepath):
        os.mkdir(output_filepath)
    
    urls = read_url()

    ChromeOptions = webdriver.ChromeOptions()
    ChromeOptions.add_argument('--disable-browser-side-navigation')
    driver = webdriver.Chrome()
    driver.set_page_load_timeout(30)
    
    for index, url in enumerate(urls):
        try:
            print("Progress - {}/{}".format(index+1,len(urls)), end="\r")
            tab_in_focus = 0
            visited_links = []
            visited_links.append(url)
            driver.get(url)
            re_scrape(driver, 1)
            with open("{}/{}.json".format(output_filepath, index), 'w') as fout:
                json.dump(product_list, fout, indent=4)
        except Exception as e:
            print(e)
    driver.quit()
    print("Finished")
            

Enter links filepath : links.txt
Enter output folder : sdf
Toys https://www.flipkart.com/toys/pr?sid=mgl&otracker=categorytree
Remote Control Toys https://www.flipkart.com/toys/remote-control-toys/pr?sid=mgl,56m&otracker=categorytree
Learning & Educational Toys https://www.flipkart.com/toys/educational-toys/learning-toys/pr?sid=mgl%2Cu35&otracker=categorytree
Soft Toys https://www.flipkart.com/toys/soft-toys/pr?sid=mgl,1zt&otracker=categorytree
Puzzles https://www.flipkart.com/toys/puzzles/pr?sid=mgl,lu4&otracker=categorytree
Art & Craft Toys https://www.flipkart.com/toys/educational-toys/art-craft-kits/pr?sid=mgl%2C3wb&otracker=categorytree
Outdoor Toys https://www.flipkart.com/toys/outdoor-toys/pr?sid=mgl,1k9&otracker=categorytree
Action Figures https://www.flipkart.com/toys/vehicle-action-toys/action-figures/pr?sid=mgl%2Ck00&otracker=categorytree
Musical Instruments & Toys https://www.flipkart.com/toys/musical-instruments-toys/pr?sid=mgl,pz4&otracker=categorytree
Music Boxes https:/

Sports https://www.flipkart.com/toys/sports/pr?sid=mgl,v1n&otracker=categorytree
Tennis https://www.flipkart.com/toys/sports/tennis/pr?sid=mgl,v1n,0ms&otracker=categorytree
Football https://www.flipkart.com/toys/sports/football/pr?sid=mgl,v1n,u74&otracker=categorytree
Cricket https://www.flipkart.com/toys/sports/cricket/pr?sid=mgl,v1n,g1o&otracker=categorytree
Role Play Toys https://www.flipkart.com/toys/role-play-toys/pr?sid=mgl,bbl&otracker=categorytree
Spinning & Press-n-Launch Toys https://www.flipkart.com/toys/spinning-pressnlaunch-toys/pr?sid=mgl,hu3&otracker=categorytree
Toy Guns & Others https://www.flipkart.com/toys/vehicle-action-toys/battle-toys/pr?sid=mgl%2Cgsh&otracker=categorytree
Marbles https://www.flipkart.com/toys/marbles/pr?sid=mgl,qyu&otracker=categorytree
Mikado Sticks https://www.flipkart.com/toys/mikado-sticks/pr?sid=mgl,hfq&otracker=categorytree
Blocks & Building Sets https://www.flipkart.com/toys/blocks-constructions/pr?sid=mgl%2C61z&otracker=categorytree
Finis

In [ ]:

tab_in_focus = 0
url = 'https://www.flipkart.com/footwear/pr?sid=osp&otracker=categorytree'
# url = 'https://www.flipkart.com/mens-footwear/pr?sid=osp%2Ccil&otracker=categorytree'
category = 'Footwear'
Thread(target=write_to_file, daemon=True).start()
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('--disable-browser-side-navigation')
driver = webdriver.Chrome()
# driver = webdriver.Firefox()
driver.set_page_load_timeout(30)
driver.get(url)
re_scrape(driver)
print("Finish")
# driver.quit()

In [ ]:
url = "https://www.flipkart.com/footwear/womens-footwear/pr?sid=osp,iko&otracker=categorytree"
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('--disable-browser-side-navigation')
driver = webdriver.Chrome()
# driver = webdriver.Firefox()
driver.set_page_load_timeout(30)
driver.get(url)

In [ ]:
price = driver.find_elements_by_class_name("_3AsjWR")[-1].text
print(price)

In [ ]:
for item in driver.find_elements_by_class_name("ttx38n"):
    if item.get_attribute("class") == "ttx38n":
        item.click()


In [ ]:
for item in driver.find_elements_by_class_name("QvtND5")[::-1]:
    if "MORE" in item.text:
        driver.execute_script("arguments[0].click();", item)

In [ ]:
[ filter_item.text.split("\n") for filter_item in driver.find_elements_by_class_name("_2hbLCH")[1:] ]

In [ ]:
[ item.get_attribute('href') for item in driver.find_elements_by_class_name("IRpwTa") ]

In [ ]:
driver.find_element_by_class_name('_2aDURW').text.replace("CATEGORIES","").strip().split("\n")
driver.find_element_by_class_name('_1jJQdf').text
# links = []
# links.append([item.get_attribute('href') for item in driver.find_elements_by_class_name('_2SqgSY')])
# print(links)

In [ ]:
# Open a new window
driver.execute_script("window.open('');")
# Switch to the new window
driver.switch_to.window(driver.window_handles[1])

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.get('https://www.flipkart.com/Women's Footwear/pr?sid=osp&otracker=categorytree')